In [1]:
class QueueNode:
    def __init__(self, data, priority):
        self.Data = data
        self.Priority = priority
        self.Next = None

class MyQueue:

    def __init__(self,isAscending):
        self.root = None
        self.isAscending = isAscending

    def IsEmpty(self):
        return self.root is None
        
    def Enqueue(self, item, priority):
        temp = QueueNode(item, priority)
        if (self.isAscending):
            if (self.root is None):
                self.root = temp
                
            # If root has low priority, insert new node at root.
            elif (self.root.Priority > priority):
                temp.Next = self.root
                self.root = temp  
            else:
                head = self.root
                while (head.Next is not None and head.Next.Priority <= priority):
                    head = head.Next
                temp.Next = head.Next
                head.Next = temp     
        else:
            if (self.root is None):
                self.root = temp

            #If root has low priority, insert new node at root.
            elif (self.root.Priority < priority):
                temp.Next = self.root
                self.root = temp
            else:
                
                head = self.root
                while (head.Next is not None and head.Next.Priority <= priority):
                    head = head.Next
                temp.Next = head.Next
                head.Next = temp     

    def ChangePriority(self, item, priority):
        if (self.root is None):
            return None
        else:
            temp = self.root
            while(temp.Next is not None and temp.Data is not item):
                temp = temp.Next
            if (temp.Data is item):
                temp.Priority = priority
        
        return None
    
    def HasItem(self, item):
        if (self.root is None):
            return False
        else:
            temp = self.root
            while(temp.Next is not None and temp.Data is not item):
                temp = temp.Next
            if (temp.Data is item):
                return True
        
        return False
    
    def GetPriority(self, item):
        if (self.root is None):
            return None
        else:
            temp = self.root
            while(temp.Next is not None and temp.Data is not item):
                temp = temp.Next
            if (temp.Data is item):
                return temp.Priority
        
        return None
        

    def Dequeue(self):
        if (self.root is None):
            print("Queue is Empty")
        else:
            item = self.root
            self.root = self.root.Next
            return item
        
        return None

    def Peek(self):
        if (self.root is not None):
            return self.root
        
        return None
    

In [2]:
class Graph:

    def __init__(self):
        self.graph = {}
        self.mst = {}

    def AddEdge(self,u, v, w):
        if(u not in self.graph):
            self.graph[u] = []
        if(v not in self.graph):
            self.graph[v] = []

        self.graph[u].append((v, w))
        self.graph[v].append((u, w))
    
    def AddDummy(self,u,v, w):
        self.AddEdge(u,'DUMMY',w)
        self.AddEdge(v,'DUMMY',w)
    
    def IsAdjacent(self, u, v):
        for m,w in self.graph[v]:
            if m is u:
                return True
            
        return False

    def GetVertices(self):
        return "".join(list(self.graph.keys()))
    
    def getWeight(self, u,v):
        for m,w in self.graph[v]:
            if m is u:
                return w
            
        return 0
    
    def ComputeCost(self, path):
        sum = 0
        for i in range(len(path) - 1):
            sum +=self.getWeight(path[i], path[i+1])
        return sum
    
gr = Graph()  
gr.AddEdge('A', 'B', 5)
gr.AddEdge('A', 'C', 8)
gr.AddEdge('B', 'C', 7)
gr.AddEdge('B', 'D', 6)
gr.AddEdge('B', 'E', 10)
gr.AddEdge('B', 'H', 8)
gr.AddEdge('C', 'F', 12)
gr.AddEdge('D', 'H', 10)
gr.AddEdge('E', 'F', 9)
gr.AddEdge('E', 'H', 18)

# gr.AddEdge('A', 'B', 5)
# gr.AddEdge('A', 'C', 2)
# gr.AddEdge('A', 'D', 3)
# gr.AddEdge('B', 'C', 6)
# gr.AddEdge('B', 'D', 3)
# gr.AddEdge('C', 'D', 4)

print(gr.graph)


{'A': [('B', 5), ('C', 8)], 'B': [('A', 5), ('C', 7), ('D', 6), ('E', 10), ('H', 8)], 'C': [('A', 8), ('B', 7), ('F', 12)], 'D': [('B', 6), ('H', 10)], 'E': [('B', 10), ('F', 9), ('H', 18)], 'H': [('B', 8), ('D', 10), ('E', 18)], 'F': [('C', 12), ('E', 9)]}


In [3]:
class GoalState:
    def __init__(self, graphObject, src):
        self.graphObject = graphObject
        self.src = src

    def IsGoal(self, parents, n):
        path = self.GetClosedPath(parents,n)
        if len(path)==len(self.graphObject.graph):
            print(path)
            print(self.graphObject.ComputeCost(path))
            return 1
        else:
            return 0

    def GetClosedPath(self, parents, n):
        path = []
        while parents[n] != n:
            path.append(n)
            n = parents[n]
        path.append(self.src)
        path.reverse()
        return path


In [4]:
import copy
class MST_Heuristic:
    def __init__(self, graph):
        self.originalGraph = graph
        self.mstDict = {}
    
    #Get the MST using PRIMS
    def MST(self,graph):
        #Create a priority queue to store vertices that are being preprocessed.
        pq = MyQueue(True)

        #Insert source itself in priority queue and initialize its distance as 0.
        if (len(graph.keys()) <= 1):
            return 0
        
        source = list(graph.keys())[0]
        pq.Enqueue(source, 0)

        visited = {}
        parent = {}
        w = {}

        for v in graph:
            w[v] = 10000
            visited[v] = 0

        sum = 0
            
        # Looping till priority queue becomes empty (or all distances are not finalized)
        while (pq.root is not None):
            # The first vertex in pair is the minimum distance
            # vertex, extract it from priority queue.
            # vertex label is data and priority is distance.
            min = pq.Dequeue()
            u = min.Data

            if (u in visited and visited[u] == 1):
                continue
            
            visited[u] = 1
            w[u] = min.Priority 

            if(u in parent and u in w):
                #print(parent[u] + " - " + u + " " + str(w[u]))
                sum += w[u]

            for v,weight in graph[u]:
                if (visited[v] != 1 and w[v] > weight):
                    w[v] = weight
                    pq.Enqueue(v, weight)
                    parent[v] = u
        return sum

    def RemoveNodesFromGraph(self, graph, nodesToRemove):
        for i in nodesToRemove:
            graph.pop(i)
        
        for v in graph:
            for u,w in graph[v].copy():
                if u in nodesToRemove:
                    graph[v].remove((u,w))

    def GetHeuristic(self, source, visitedNodes):
        mstDistance = 0
        mstGraph = Graph()
        mstGraph.graph = copy.deepcopy(self.originalGraph)
        self.RemoveNodesFromGraph(mstGraph.graph, visitedNodes)
        vertices = mstGraph.GetVertices()
        if vertices in self.mstDict:
            mstDistance = self.mstDict[vertices]
        else:
            mstDistance = self.MST(mstGraph.graph)
            self.mstDict[vertices] =  mstDistance
        
        minSourceToMST = 10000
        minLastVisitedToMST = 10000
        for v in self.originalGraph:
            if v not in visitedNodes:
                for u,w in self.originalGraph[v]:
                    if u is source and minSourceToMST > w:
                        minSourceToMST = w
                    if u is visitedNodes[-1] and minLastVisitedToMST > w:
                        minLastVisitedToMST = w

        return mstDistance + minSourceToMST + minLastVisitedToMST

In [10]:
class AStar_TSP:
    def __init__(self, graphObject, start_node, end_node):
        self.heuristic = MST_Heuristic(graphObject.graph)
        self.graphObject = graphObject
        self.start_node = start_node
        self.end_node = end_node
        self.goalState = GoalState(graphObject, start_node)

    def aStar_tsp(self):
        open_set_cost_dict = {}
        open_set_cost_dict[self.start_node] = self.heuristic.GetHeuristic(self.start_node, self.start_node)
        closed_set = set()
        g = {}                   # store distance from starting node
        parents = {}             # parents contains parent information of visited nodes  
        g[self.start_node] = 0   #distance of starting node from itself is zero
        parents[self.start_node] = self.start_node #The parent of the start node is itself
        goalState = 0
        while not goalState and len(open_set_cost_dict) > 0:
            n = list(open_set_cost_dict.keys())[0]      
            minimumCost = open_set_cost_dict[list(open_set_cost_dict.keys())[0]] 
            # Pick node having min f_value from the fringe list
            for v in open_set_cost_dict:
                if minimumCost > open_set_cost_dict[v]:
                    minimumCost = open_set_cost_dict[v]
                    n = v
            h = open_set_cost_dict[n]	
            path = self.goalState.GetClosedPath(parents,n)
            closed_set = set(path)
            if self.goalState.IsGoal(parents, n):
                goalState = 1
                pass
        
            open_set_cost_dict = {}
            for j in self.graphObject.graph:
                if not j in closed_set and self.graphObject.getWeight(j,n) != 0:
                    if j is self.end_node and len(path) != len(self.graphObject.graph) - 1:
                        continue
                    visitedNodes = list(closed_set)
                    visitedNodes.append(j)
                    h = self.heuristic.GetHeuristic(j, visitedNodes)

                    g[j] = g[n] + self.graphObject.getWeight(j,n) + h
                    parents[j] = n
                    open_set_cost_dict[j] = g[j]
        
        if not goalState:
            print("No path is found!")
            return None  
        else:
            return self.graphObject.ComputeCost(path)


gr = Graph()
gr.AddEdge('A', 'B', 5)
gr.AddEdge('A', 'C', 8)
gr.AddEdge('B', 'C', 7)
gr.AddEdge('B', 'D', 6)
gr.AddEdge('B', 'E', 10)
gr.AddEdge('B', 'H', 8)
gr.AddEdge('C', 'F', 12)
gr.AddEdge('D', 'H', 10)
gr.AddEdge('E', 'F', 9)
gr.AddEdge('E', 'H', 18)

# gr.AddEdge('A', 'B', 5)
# gr.AddEdge('A', 'C', 2)
# gr.AddEdge('A', 'D', 3)
# gr.AddEdge('B', 'C', 6)
# gr.AddEdge('B', 'D', 3)
# gr.AddEdge('C', 'D', 4)

print(gr.graph)  

a = AStar_TSP(gr, 'A', 'H')
a.aStar_tsp()


{'A': [('B', 5), ('C', 8)], 'B': [('A', 5), ('C', 7), ('D', 6), ('E', 10), ('H', 8)], 'C': [('A', 8), ('B', 7), ('F', 12)], 'D': [('B', 6), ('H', 10)], 'E': [('B', 10), ('F', 9), ('H', 18)], 'H': [('B', 8), ('D', 10), ('E', 18)], 'F': [('C', 12), ('E', 9)]}
['A', 'C', 'F', 'E', 'B', 'D', 'H']
55


55